In [1]:
import os
import numpy as np
import pandas as pd

import geopandas as gpd
import rioxarray as rioxr
from random import sample

from shapely.geometry import Point 

from rasterio.crs import CRS

In [2]:
# Load raster
year = 2020

root = '/home/jovyan/msai4earth-esa/iceplant_detection/temp/' + os.path.join('LS_filter_clip_preds_' + str(year))

fp = os.path.join(root, 'LS_merged_crs26910_S_'+str(year)+'.tif')
r_26910_S = rioxr.open_rasterio(fp).squeeze()

fp = os.path.join(root, 'LS_merged_crs26910_W_'+str(year)+'.tif')
r_26910_W = rioxr.open_rasterio(fp).squeeze()

fp = os.path.join(root, 'LS_merged_crs26911_'+str(year)+'_clip.tif')
r_26911 = rioxr.open_rasterio(fp).squeeze()

rasters = [r_26910_S, r_26910_W, r_26911]

In [3]:
pixel_count = pd.read_csv(os.path.join(os.getcwd(), 'rasters_2020_pixel_counts.csv'))
pixel_count

,n_nonice_20,n_ice_20,n_ground_20,n_water_20,raster
0,33222542,9359285,113802445,126328228,LS_merged_crs26910_S_2020
1,1005498,626950,3058661,6139494,LS_merged_crs26910_W_2020
2,89606257,2915223,63428879,138290707,LS_merged_crs26911_2020_clip


In [4]:
# STRATIFIED RANDOM SAMPLING
cats = [0, 1, 2, 3]
#samples_per_class = [87, 87, 91, 135]
samples_per_class = [3]*4
dist = []

for c in cats:
    
    # pixels in each raster from specified class
    n1 = list(pixel_count.iloc[:,c])[0]
    n2 = list(pixel_count.iloc[:,c])[1]
    n3 = list(pixel_count.iloc[:,c])[2]
    s = n1 + n2 + n3
     
    # randomly specify number of samples per raster
    where_to_sample = np.random.choice([0,1,2], size=samples_per_class[c], p=[n1/s, n2/s, n3/s])

    dist.append([len(np.where(where_to_sample == 0)[0]), 
                 len(np.where(where_to_sample == 1)[0]), 
                 len(np.where(where_to_sample == 2)[0])])
    
dist

[[2, 0, 1], [3, 0, 0], [3, 0, 0], [1, 0, 2]]

In [6]:
all_points = []

for i in range(0,3):

    points = []
    which_class = []
    which_raster = []
    sampled_raster = False

    for c in cats:
        n_samples = dist[c][i]

        if n_samples != 0:
            sampled_raster = True

            raster = rasters[i]

            # arrays with indices where condition is True
            c_raster = np.where(raster == c)

            # sample random (y,x) pairs from list
            indices = sample(list(zip(c_raster[0], c_raster[1])), n_samples)

            # unzip into y and x coordinates
            y, x = list(zip(*indices))

            # x and y coordinates in raster CRS corresponding to sampled points
            x_coord = raster.x[np.asarray(x)]
            y_coord = raster.y[np.asarray(y)]

            # make points 
            points = points + [Point(m,n) for m,n in zip(x_coord, y_coord)]

            which_class = which_class + [c]*n_samples
            which_raster = which_raster + [i]*n_samples

    if sampled_raster:
        points_df = gpd.GeoDataFrame({'geometry': points,
                                      'class' : which_class,
                                      'which_raster' : which_raster},
                                     crs = raster.rio.crs)
        all_points.append(points_df.to_crs(CRS.from_epsg(4326)))  # change to lat/lon crs
        


In [17]:
points_df = pd.concat(all_points, ignore_index=True)
points_df['LON'] = points_df.geometry.x
points_df['LAT'] = points_df.geometry.y
points_df = points_df.drop(['geometry'], axis=1)

In [18]:
points_df

,class,which_raster,LON,LAT
0,0,0,-120.400404,34.450079
1,0,0,-120.232938,34.476533
2,1,0,-120.212521,34.471655
3,1,0,-120.173940,34.470601
4,1,0,-120.146163,34.480864
5,2,0,-120.095638,34.469348
6,2,0,-120.391666,34.460402
7,2,0,-120.000856,34.463050
8,3,0,-120.339235,34.454874
9,0,2,-119.758322,34.408429


In [19]:
points_df.to_csv('tiny_sample.csv', index_label='PLOTID')